# Settings

In [24]:
OUTPUT_CLEANED_FOLDER = "../notebooks/output/"
DATA_NCF = "data/"

# Read data

In [2]:
import os 
import pandas as pd 

df_interaction = pd.read_csv(os.path.join(OUTPUT_CLEANED_FOLDER, "user_activity_interaction_deep_cleaned.csv"))
df_user = pd.read_csv(os.path.join(OUTPUT_CLEANED_FOLDER, "user_info_deep_cleaned.csv"))
df_activity = pd.read_csv(os.path.join(OUTPUT_CLEANED_FOLDER, "activity_info_deep_cleaned.csv"))

/tmp/ipykernel_3990194/1144630671.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_interaction = pd.read_csv(os.path.join(OUTPUT_CLEANED_FOLDER, "user_activity_interaction_deep_cleaned.csv"))


In [3]:
df_activity.dtypes

id               int64
title           object
type            object
desc            object
btc             object
createdTime     object
startTime       object
finishTime      object
deadlineTime    object
criteria        object
dtype: object

In [4]:
df_activity["id"]  = df_activity["id"].astype(str)

In [5]:
df_activity.head()

,id,title,type,desc,btc,createdTime,startTime,finishTime,deadlineTime,criteria
0,135,Tọa đàm đối thoại giữa sinh viên và lãnh đạo n...,Sinh hoạt chuyên đề,"Các bạn sinh viên thân mến, Gần đây, Trường ch...",Đại học Bách Khoa Hà Nội,2019-11-26 18:03:25,2019-11-26 18:00:00,2019-11-27 21:00:00,2019-11-27 21:00:00,"[{'CGroupId': 0, 'CId': 67, 'CName': 'Tham gia..."
1,136,Ngày hội tuyển dụng của công ty LG Electronics,Hội thảo hướng nghiệp,Công ty LG Electronics Việt Nam Hải Phòng (LGE...,Ban CTSV,2019-11-27 14:51:13,2019-11-29 08:30:00,2019-11-29 11:30:00,2019-11-29 11:30:00,"[{'CGroupId': 0, 'CId': 61, 'CName': 'Tham gia..."
2,137,Cổ vũ Giải bóng đá nam sinh viên các trường...,Hoạt động thể thao,Giải bóng đá sinh viên Hà Nội là giải đấu thườ...,Ban CTSV,2019-12-02 11:16:37,2019-12-02 08:00:00,2019-12-02 10:30:00,2019-12-02 10:30:00,"[{'CGroupId': 0, 'CId': 62, 'CName': 'Tham gia..."
3,146,"Lễ tổng kết cuộc thi ""Tuổi trẻ học tập và làm ...",Sinh hoạt chuyên đề,- Tiếp tục đẩy mạnh học tập và làm theo tư tưở...,Ban CTSV,2019-12-11 09:37:19,2019-12-14 18:00:00,2019-12-14 22:00:00,2019-12-14 22:00:00,"[{'CGroupId': 0, 'CId': 60, 'CName': 'Tham gia..."
4,147,Họp Chi bộ 12/2019,Hiến máu nhân đạo,Kiểm điểm đảng viên Triển khai công tác khác c...,Chi bộ SV II,2019-12-11 14:25:48,2019-12-08 17:00:00,2019-12-08 19:30:00,2019-12-08 19:30:00,"[{'CGroupId': 0, 'CId': 55, 'CName': 'Tham gia..."


In [6]:
df_interaction = df_interaction.astype(str)
df_interaction.dtypes

user_id            object
activity_id        object
accepted_status    object
dtype: object

In [7]:
df_interaction.head()

,user_id,activity_id,accepted_status
0,20186237,135,2
1,20170351,135,2
2,20191797,135,2
3,20174071,135,1
4,20193457,135,1


In [8]:
df_interaction["inter_score"] = 1

In [9]:
df_interaction

,user_id,activity_id,accepted_status,inter_score
0,20186237,135,2,1
1,20170351,135,2,1
2,20191797,135,2,1
3,20174071,135,1,1
4,20193457,135,1,1
...,...,...,...,...
3287967,20230352P,8969,2,1
3287968,20210381P,8969,2,1
3287969,20210126P,8969,2,1
3287970,20230327P,8969,2,1


In [10]:
df_all = df_interaction.merge(df_activity, left_on=["activity_id"], right_on=["id"])[["user_id", "activity_id", "inter_score", "createdTime"]]
df_all.head()

,user_id,activity_id,inter_score,createdTime
0,20186237,135,1,2019-11-26 18:03:25
1,20170351,135,1,2019-11-26 18:03:25
2,20191797,135,1,2019-11-26 18:03:25
3,20174071,135,1,2019-11-26 18:03:25
4,20193457,135,1,2019-11-26 18:03:25


In [11]:
df_all.tail()

,user_id,activity_id,inter_score,createdTime
3287967,20230352P,8969,1,2023-11-23 10:48:28
3287968,20210381P,8969,1,2023-11-23 10:48:28
3287969,20210126P,8969,1,2023-11-23 10:48:28
3287970,20230327P,8969,1,2023-11-23 10:48:28
3287971,20234387,8870,1,2023-10-17 12:12:00


In [13]:
df_all["createdTime"] = pd.to_datetime(df_all["createdTime"])

# Training Testing Split


In [14]:
def train_test_split_time(df , columns_1 , columns_2 , columns_time):
    '''
    we will use the columns_1 as the first level index
    sort the columns_2 as the target by columns_time
    '''
    
    retrieve_name = df[columns_1].value_counts()[df[columns_1].value_counts() > 5].keys()
    df_test = df[df[columns_1].isin(retrieve_name)]
    df_test = df_test.sort_values(columns_time).groupby(columns_1).tail(1)
    df_train = df.drop(index=df_test.index)
    
    if df_test.shape[0] + df_train.shape[0] == df.shape[0]:
        print('train_test_split succeed!! with df_train shape:(%d,%d), df_test shape:(%d,%d)'
              %(df_train.shape[0],df_train.shape[1],df_test.shape[0],df_test.shape[1])
             )
        return df_train, df_test, retrieve_name
    else:
        print('Oops, something wrong, with df_train shape : (%d,%d), df_test shape : (%d,%d)'
              %(df_train.shape[0],df_train.shape[1],df_test.shape[0],df_test.shape[1])
             )
        print('df_shape : (%d,%d)'
              %(df.shape[0],df.shape[1])
             )

In [15]:
df_train, df_test, retrieve = train_test_split_time(df_all, 'user_id','activity_id', 'createdTime')


train_test_split succeed!! with df_train shape:(3234660,4), df_test shape:(53312,4)


In [16]:
df_train[df_train["user_id"] == "20204876"].sort_values(by="createdTime")

,user_id,activity_id,inter_score,createdTime
449583,20204876,901,1,2020-09-18 11:32:04
1638608,20204876,907,1,2020-10-08 11:34:17
1286422,20204876,911,1,2020-10-14 17:41:22
2458868,20204876,921,1,2020-10-18 21:54:32
2838495,20204876,929,1,2020-10-22 16:43:32
...,...,...,...,...
3156415,20204876,8978,1,2023-11-25 21:58:12
2773923,20204876,8982,1,2023-11-27 08:40:36
2418474,20204876,9025,1,2023-12-08 15:44:03
2425039,20204876,9033,1,2023-12-11 16:23:32


In [17]:
df_test[df_test["user_id"] == "20204876"]

,user_id,activity_id,inter_score,createdTime
2803899,20204876,9075,1,2023-12-19 21:34:35


We will use the leave-one-out method to evaluate our testing data

Basically, we will generate N samples list that the test user hasn't read and add the ground truth into the list, and see how the recommendation system will score and rank the list

In [18]:
set(df_all["activity_id"].unique())

{'2475',
 '3342',
 '1726',
 '6553',
 '7214',
 '1464',
 '8571',
 '1249',
 '6646',
 '1307',
 '1380',
 '1211',
 '1898',
 '1499',
 '3275',
 '6561',
 '1077',
 '2606',
 '6589',
 '1039',
 '1544',
 '9039',
 '1590',
 '1124',
 '8605',
 '8901',
 '6877',
 '1068',
 '7113',
 '2671',
 '751',
 '2284',
 '7240',
 '2124',
 '1366',
 '542',
 '302',
 '2160',
 '8549',
 '1843',
 '3277',
 '2541',
 '1591',
 '2519',
 '5536',
 '2851',
 '1695',
 '2267',
 '1228',
 '2765',
 '8744',
 '8777',
 '8825',
 '3029',
 '3220',
 '3176',
 '1469',
 '1840',
 '8400',
 '2799',
 '9012',
 '3282',
 '2559',
 '8646',
 '2341',
 '1897',
 '435',
 '2353',
 '3009',
 '1900',
 '3356',
 '9084',
 '8677',
 '2979',
 '2214',
 '8716',
 '7099',
 '6733',
 '1347',
 '6958',
 '9073',
 '6861',
 '6857',
 '6750',
 '8999',
 '1140',
 '6614',
 '879',
 '2664',
 '704',
 '6795',
 '6737',
 '8648',
 '7219',
 '1597',
 '8446',
 '8388',
 '6610',
 '436',
 '2176',
 '8801',
 '8872',
 '6928',
 '8851',
 '220',
 '3105',
 '1942',
 '789',
 '1071',
 '3122',
 '3236',
 '2283',
 

In [19]:
len(set(df_all["activity_id"].unique()).difference(sorted(df_all.groupby("user_id")['activity_id'].apply(list)["20204876"])))

4310

In [20]:
import random

random.seed(73)

def get_sampling(df_target, df_source, numbers_of_N_sample, generate_negative = False):
    '''
    1.
    return: test_rating in the shape of [ [user_pid,article] * N ]
    2.
    from the target dataframe, sampling the negative sample from source dataframe
    return: list in the shape of [ [((user_id, ground truth)) , [numbers_of_N_sample(negative samples)]] * N]
    '''
    test_rating = df_target[['user_id','activity_id']].values.tolist()
    print('test_rating yield successfully!!!')
    
    if generate_negative:
        act_set = set(df_source["activity_id"].unique())
        mapping_user_act = df_source.groupby("user_id")['activity_id'].apply(list)
        negative_test_rating = []
        for i in range(df_target.shape[0]):
            drop_id = test_rating[i][0]
            list_ = random.sample(list(act_set.difference(mapping_user_act[drop_id])), numbers_of_N_sample)
            negative_test_rating.append([test_rating[i], list_])
        print('negative_test_rating yield successfully!!!')

        return test_rating, negative_test_rating
    else:
        return test_rating

In [21]:
print('''only for generating sample, if there is a sample file already, skip this cell in the future''')
# In the future just execute the next cell to get the test rating & negative sampling
# import time
# import pickle

# time1 = time.time()
# test_rating, negative_test_rating = get_sampling(df_test, df_train, 19, True)
# time2 = time.time()
# print('Took for %d seconds' %(time2-time1))
# with open(os.path.join(OUTPUT_CLEANED_FOLDER, "negative_test_rating_19N.pkl"),"wb") as f: #in write mode
#     pickle.dump(negative_test_rating, f)

only for generating sample, if there is a sample file already, skip this cell in the future


In [22]:
import time
import pickle

time1 = time.time()
test_rating = get_sampling(df_test, df_train, None)
with open(os.path.join(OUTPUT_CLEANED_FOLDER, "negative_test_rating_19N.pkl"),'rb') as f: #in read mode, not in write mode, careful
    negative_test_rating = pickle.load(f)
    print('negative_test_rating yield successfully!!!')
    
time2 = time.time()
print('Took for %d seconds' %(time2-time1))

test_rating yield successfully!!!
negative_test_rating yield successfully!!!
Took for 0 seconds


In [23]:
negative_test_rating

[[['20152751', '174'],
  ['5473',
   '8263',
   '2391',
   '8551',
   '675',
   '9009',
   '2119',
   '965',
   '466',
   '708',
   '690',
   '832',
   '388',
   '8855',
   '1667',
   '6972',
   '6827',
   '2651',
   '7063']],
 [['20153904', '280'],
  ['8706',
   '2855',
   '1021',
   '8666',
   '1351',
   '3358',
   '1608',
   '1792',
   '2217',
   '3050',
   '3192',
   '8428',
   '797',
   '8924',
   '2192',
   '1773',
   '7045',
   '8908',
   '8331']],
 [['20175850', '280'],
  ['2182',
   '2100',
   '508',
   '2652',
   '5534',
   '655',
   '8382',
   '8728',
   '3233',
   '2029',
   '474',
   '6644',
   '1196',
   '917',
   '563',
   '5521',
   '809',
   '2961',
   '9082']],
 [['20151114', '305'],
  ['281',
   '8819',
   '7000',
   '1343',
   '734',
   '7037',
   '5478',
   '1845',
   '429',
   '9040',
   '606',
   '2365',
   '8396',
   '6637',
   '1436',
   '8909',
   '2102',
   '2044',
   '6622']],
 [['20175902', '333'],
  ['1799',
   '2088',
   '1149',
   '2223',
   '2078',
   '

# Preprocessing

In [25]:
user_id_map = {i:v for i, v in enumerate(df_all["user_id"].unique())}
inverse_user_id_map = {v:i for i, v in enumerate(df_all["user_id"].unique())}
activity_id_map = {i:v for i, v in enumerate(df_all["activity_id"].unique())}
inverse_activity_id_map = {v:i for i, v in enumerate(df_all["activity_id"].unique())}

In [26]:
df_train_map = df_train.copy()
df_test_map = df_test.copy()

df_train_map['user_id'] = df_train_map['user_id'].map(inverse_user_id_map)
df_train_map['activity_id'] = df_train_map['activity_id'].map(inverse_activity_id_map)
df_test_map['user_id'] = df_test_map['user_id'].map(inverse_activity_id_map)
df_test_map['activity_id'] = df_test_map['activity_id'].map(inverse_activity_id_map)

### creating a mapping list for testing data   
test_rating_map = []
for i in range(len(test_rating)):
    test_rating_map.append([inverse_user_id_map[test_rating[i][0]],inverse_activity_id_map[test_rating[i][1]]])


### creating a mapping list for negative testing data
negative_test_rating_map = []
for i in range(len(negative_test_rating)):
    negative_test_rating_map.append([
                                     [inverse_user_id_map[negative_test_rating[i][0][0]],
                                      inverse_activity_id_map[negative_test_rating[i][0][1]]],
                                     list(map(lambda w: inverse_activity_id_map[w], negative_test_rating[i][1]))])


In [27]:
print('df_train_group_map shape: ',df_train_map.shape)
print('number of users: ', len(user_id_map))
print('number of items: ', len(activity_id_map))
print('The sparse matrix is one with shape (%d , %d), with %d non-zero read_times'
      %(len(user_id_map),len(activity_id_map), df_train_map.shape[0]))


df_train_group_map shape:  (3234660, 4)
number of users:  59272
number of items:  4505
The sparse matrix is one with shape (59272 , 4505), with 3234660 non-zero read_times


# Saving the data

In [30]:
import pickle

with open(os.path.join(DATA_NCF, "inverse_user_id_map.pkl"), "wb") as f:
    pickle.dump(inverse_user_id_map, f)
with open(os.path.join(DATA_NCF, "inverse_activity_id_map.pkl"), "wb") as f:
    pickle.dump(inverse_activity_id_map, f)
with open(os.path.join(DATA_NCF, "negative_test_rating_map.pkl"), "wb") as f:
    pickle.dump(negative_test_rating_map, f)
with open(os.path.join(DATA_NCF, "df_train_map.pkl"), "wb") as f:
    pickle.dump(df_train_map, f)
with open(os.path.join(DATA_NCF, "df_test_map.pkl"), "wb") as f:
    pickle.dump(df_test_map, f)


In [31]:
df_train_map

,user_id,activity_id,inter_score,createdTime
0,0,0,1,2019-11-26 18:03:25
1,1,0,1,2019-11-26 18:03:25
2,2,0,1,2019-11-26 18:03:25
3,3,0,1,2019-11-26 18:03:25
4,4,0,1,2019-11-26 18:03:25
...,...,...,...,...
3287967,59268,4503,1,2023-11-23 10:48:28
3287968,59269,4503,1,2023-11-23 10:48:28
3287969,59270,4503,1,2023-11-23 10:48:28
3287970,59271,4503,1,2023-11-23 10:48:28


In [32]:
with open(os.path.join(DATA_NCF, "df_train_map.pkl"), "rb") as f:
    data  = pickle.load(f)[]


In [44]:
data[["user_id", "activity_id"]].values.tolist()

[[0, 0],
 [1, 0],
 [2, 0],
 [3, 0],
 [4, 0],
 [5, 0],
 [6, 0],
 [7, 0],
 [8, 0],
 [9, 0],
 [10, 0],
 [11, 0],
 [12, 0],
 [13, 0],
 [14, 0],
 [15, 0],
 [16, 0],
 [17, 0],
 [18, 0],
 [19, 0],
 [20, 0],
 [21, 0],
 [22, 0],
 [23, 0],
 [24, 0],
 [25, 0],
 [26, 0],
 [27, 0],
 [28, 0],
 [29, 0],
 [30, 0],
 [31, 0],
 [32, 0],
 [33, 0],
 [34, 0],
 [35, 0],
 [36, 0],
 [37, 0],
 [38, 0],
 [39, 0],
 [40, 0],
 [41, 0],
 [42, 0],
 [43, 0],
 [44, 0],
 [45, 0],
 [46, 0],
 [47, 0],
 [48, 0],
 [49, 0],
 [50, 0],
 [51, 0],
 [52, 0],
 [53, 0],
 [54, 0],
 [55, 0],
 [56, 0],
 [57, 0],
 [58, 0],
 [59, 0],
 [60, 0],
 [61, 0],
 [62, 0],
 [63, 0],
 [64, 0],
 [65, 0],
 [66, 0],
 [67, 0],
 [68, 0],
 [69, 0],
 [70, 0],
 [71, 0],
 [72, 0],
 [73, 0],
 [74, 0],
 [75, 0],
 [76, 0],
 [77, 0],
 [78, 0],
 [79, 0],
 [80, 0],
 [81, 0],
 [82, 0],
 [83, 0],
 [84, 0],
 [85, 0],
 [86, 0],
 [87, 0],
 [88, 0],
 [89, 0],
 [90, 0],
 [91, 0],
 [92, 0],
 [93, 0],
 [94, 0],
 [95, 0],
 [96, 0],
 [97, 0],
 [98, 0],
 [99, 0],
 [100, 0],

In [38]:
df_train

,user_id,activity_id,inter_score,createdTime
0,20186237,135,1,2019-11-26 18:03:25
1,20170351,135,1,2019-11-26 18:03:25
2,20191797,135,1,2019-11-26 18:03:25
3,20174071,135,1,2019-11-26 18:03:25
4,20193457,135,1,2019-11-26 18:03:25
...,...,...,...,...
3287967,20230352P,8969,1,2023-11-23 10:48:28
3287968,20210381P,8969,1,2023-11-23 10:48:28
3287969,20210126P,8969,1,2023-11-23 10:48:28
3287970,20230327P,8969,1,2023-11-23 10:48:28


In [45]:
negative_test_rating_map

[[[706, 666],
  [450,
   2079,
   2224,
   4221,
   2962,
   4428,
   269,
   1178,
   2440,
   861,
   70,
   2888,
   706,
   3910,
   1136,
   2573,
   4040,
   2429,
   2158]],
 [[6629, 4],
  [3752,
   1877,
   1169,
   3391,
   891,
   1613,
   1205,
   746,
   3859,
   4115,
   3728,
   3915,
   2704,
   2655,
   1942,
   1057,
   2396,
   188,
   2598]],
 [[7963, 4],
  [353,
   350,
   1366,
   3139,
   4014,
   1408,
   3061,
   3891,
   1612,
   4161,
   1707,
   2712,
   2165,
   333,
   1982,
   880,
   1808,
   3382,
   3847]],
 [[6528, 857],
  [784,
   4380,
   2087,
   87,
   1662,
   1030,
   1534,
   3828,
   786,
   1973,
   2047,
   570,
   2923,
   3589,
   3468,
   3793,
   1467,
   3386,
   2934]],
 [[8588, 2410],
  [2782,
   3135,
   1580,
   694,
   3043,
   747,
   709,
   3049,
   2045,
   3710,
   1961,
   4290,
   1752,
   3748,
   2843,
   2755,
   3334,
   4061,
   1692]],
 [[58021, 2554],
  [1890,
   2870,
   2397,
   884,
   4415,
   4350,
   3243,
   121

In [46]:
test_data = []
for data in negative_test_rating_map:
		u = data[0][0]
		test_data.append(data[0])
		for neg in data[1]:
			test_data.append([u, neg])


In [47]:
test_data

[[706, 666],
 [706, 450],
 [706, 2079],
 [706, 2224],
 [706, 4221],
 [706, 2962],
 [706, 4428],
 [706, 269],
 [706, 1178],
 [706, 2440],
 [706, 861],
 [706, 70],
 [706, 2888],
 [706, 706],
 [706, 3910],
 [706, 1136],
 [706, 2573],
 [706, 4040],
 [706, 2429],
 [706, 2158],
 [6629, 4],
 [6629, 3752],
 [6629, 1877],
 [6629, 1169],
 [6629, 3391],
 [6629, 891],
 [6629, 1613],
 [6629, 1205],
 [6629, 746],
 [6629, 3859],
 [6629, 4115],
 [6629, 3728],
 [6629, 3915],
 [6629, 2704],
 [6629, 2655],
 [6629, 1942],
 [6629, 1057],
 [6629, 2396],
 [6629, 188],
 [6629, 2598],
 [7963, 4],
 [7963, 353],
 [7963, 350],
 [7963, 1366],
 [7963, 3139],
 [7963, 4014],
 [7963, 1408],
 [7963, 3061],
 [7963, 3891],
 [7963, 1612],
 [7963, 4161],
 [7963, 1707],
 [7963, 2712],
 [7963, 2165],
 [7963, 333],
 [7963, 1982],
 [7963, 880],
 [7963, 1808],
 [7963, 3382],
 [7963, 3847],
 [6528, 857],
 [6528, 784],
 [6528, 4380],
 [6528, 2087],
 [6528, 87],
 [6528, 1662],
 [6528, 1030],
 [6528, 1534],
 [6528, 3828],
 [6528, 7